# **GIT SETUP**

In [1]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)

/content/drive
Mounted at /content/drive


**Change to your custom path**

In [2]:
MY_GOOGLE_DRIVE_PATH = 'My Drive/Capstone_Pritam/'

**Setup directories**

In [3]:

GIT_TOKEN = '4f61cb936944e66abd13e7e809e199ee3a378fae'
GIT_USERNAME = 'mohitgulla' 
GIT_REPOSITORY = 'Edge'

from os.path import join  

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# # In case we haven't created the folder already; we will create a folder in the project path 
!mkdir -p "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/My Drive/Capstone_Pritam/
GIT_PATH:  https://4f61cb936944e66abd13e7e809e199ee3a378fae@github.com/mohitgulla/Edge.git


**Change dirs and clone repo (if needed !)**

In [4]:
%cd "{PROJECT_PATH}"
#!git clone -b Pritam "{GIT_PATH}"
%cd "Edge"

/content/drive/My Drive/Capstone_Pritam
/content/drive/My Drive/Capstone_Pritam/Edge


# **Model = DNN, Data = MV data**

In [7]:
# %%writefile train_mv.py
import time
import torch
import torch.nn as nn
from model.dnn import DenseNeuralNet
from data.mv_data import MVDataset
from utils.util_functions import *
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def evaluate(model, test_set, batch_size, criterion, ep):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  

  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  for step, inputs in enumerate(test_iterator):
      global_step +=1
      x = inputs[0]
      y = inputs[1]
      x = x.to(device)
      y = y.to(device)

      logits = model(x)
      loss = criterion(logits, y)
      total_loss +=loss

  total_loss = total_loss / global_step 
  return total_loss


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  criterion = nn.MSELoss()
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
  
  train_log = open("log/train.log", "w")
  val_log = open("log/val.log", "w")
  test_log = open("log/test.log", "w")
  

  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    for step, inputs in enumerate(train_iterator):
      model.train()
      optimizer.zero_grad()

      global_step +=1
      x = inputs[0]
      y = inputs[1]
      x = x.to(device)
      y = y.to(device)

      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()

      
    val_loss = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {} \n".format(ep+1, val_loss))
    
    if not skip_train_set:
      train_loss  = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {} \n".format(ep+1, train_loss))
      print("Step = %d, training loss =  %f" %(global_step, train_loss))

    print("Step = %d, validation loss =  %f" %(global_step, val_loss))
    
  test_loss = evaluate(model, test_set, batch_size, criterion, ep)
  test_log.write("End of training, test loss =  {}\n".format(test_loss))
  print("End of Training, test loss =  %f" %(test_loss))

  train_log.close()
  val_log.close()
  test_log.close()

def main():
## main
  input_dim =  10
  output_classes = 1
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  ####

  churn_dataset = MVDataset()
  train_set, val_set, test_set = get_get_train_val_test(churn_dataset, val_split=0.40)


  model = DenseNeuralNet(input_dim, output_classes)



  train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)


if __name__ == "__main__":
  main()





Step = 1529, validation loss =  1.767898



Step = 3058, validation loss =  0.458944



Step = 4587, validation loss =  0.242651



Step = 6116, validation loss =  0.185463



Step = 7645, validation loss =  0.171183


...


Step = 10703, validation loss =  0.101186



Step = 12232, validation loss =  0.083129



Step = 13761, validation loss =  0.120491



Step = 15290, validation loss =  0.107923




End of Training, test loss =  0.110740
